In [214]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime
import csv

In [215]:
# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

In [216]:
# Load a sample picture and learn how to recognize it.
path='Images'
images = []
known_face_names = []
mylist = os.listdir(path)
print(mylist)

['ajith.jpg', 'arun vijay.jpg', 'karthi.jpg', 'surya.jpg', 'vijay.jpg', 'vj sedhupathi.jpg']


In [217]:
for i in mylist:
    curimg = cv2.imread(f'{path}/{i}')
    images.append(curimg)
    known_face_names.append(os.path.splitext(i)[0].upper())
known_face_names

['AJITH', 'ARUN VIJAY', 'KARTHI', 'SURYA', 'VIJAY', 'VJ SEDHUPATHI']

In [218]:
def findencodings(images):
    encodelist = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodelist.append(encode)
    return encodelist

In [219]:
def record(name):
    with open("record.csv",'r+') as f:
        mydatalist = f.readlines()
        namelist = []
        for line in mydatalist:
            entry = line.split(',')
            namelist.append(entry[0])
        if name not in namelist:
            now = datetime.now()
            dtstring = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtstring}')

In [220]:
def new_record (name):
    with open("Master record.csv","r") as mr:
            mrlist = csv.DictReader(mr)
            fieldnames = ['Car Number', 'Name', 'Age', 'Height (cm)', 'Weight (kg)', 'Blood Group', 'Last visit', 'Reason for last visit', 'Doctor Name']

            with open('New record.csv', 'w', newline="") as nr:
         
                csv_writer  = csv.DictWriter(nr, fieldnames=fieldnames, delimiter=',')

                csv_writer.writeheader()

                for line in mrlist:
                    if line['Name'] in face_list:
                        csv_writer.writerow(line)

In [221]:
known_face_encodings = findencodings(images)

In [222]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
face_list=[]
process_this_frame = True

In [223]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
               
            face_list.append(name)
            face_names.append(name)
            new_record(name)
            record(name)
            
            

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 200, 0), 2)

        # Draw a label with a name above the face
        cv2.rectangle(frame, (left, top - 35), (right, top), (0, 200, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left +6, top -6), font, 1.0, (255, 255, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()